In [1]:
import findspark
findspark.init()

In [4]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [5]:
spark.sparkContext.uiWebUrl

'http://host.docker.internal:4040'

In [30]:
df_green = spark.read.parquet('data/pq/green/*/*')

```
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
```

In [6]:
rdd = df_green \
    .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
    .rdd

In [7]:
from datetime import datetime

In [8]:
start = datetime(year=2020, month=1, day=1)

def filter_outliers(row):
    return row.lpep_pickup_datetime >= start

In [9]:
rows = rdd.take(10)
row = rows[0]

In [10]:
row

Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 12, 18, 15, 4), PULocationID=41, total_amount=7.88)

In [11]:
def prepare_for_grouping(row): 
    hour = row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
    zone = row.PULocationID
    key = (hour, zone)
    
    amount = row.total_amount
    count = 1
    value = (amount, count)

    return (key, value)

In [12]:
def calculate_revenue(left_value, right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value
    
    output_amount = left_amount + right_amount
    output_count = left_count + right_count
    
    return (output_amount, output_count)

In [13]:
from collections import namedtuple

In [14]:
RevenueRow = namedtuple('RevenueRow', ['hour', 'zone', 'revenue', 'count'])

In [15]:
def unwrap(row):
    return RevenueRow(
        hour=row[0][0], 
        zone=row[0][1],
        revenue=row[1][0],
        count=row[1][1]
    )

In [16]:
from pyspark.sql import types

In [17]:
result_schema = types.StructType([
    types.StructField('hour', types.TimestampType(), True),
    types.StructField('zone', types.IntegerType(), True),
    types.StructField('revenue', types.DoubleType(), True),
    types.StructField('count', types.IntegerType(), True)
])

In [18]:
df_result = rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwrap) \
    .toDF(result_schema) 

In [19]:
df_result.show()

+-------------------+----+------------------+-----+
|               hour|zone|           revenue|count|
+-------------------+----+------------------+-----+
|2020-01-11 21:00:00| 244| 336.4700000000001|   10|
|2020-01-19 10:00:00| 116|            438.91|   22|
|2020-01-28 08:00:00| 121|             45.11|    2|
|2020-01-10 22:00:00| 116|             73.79|   11|
|2020-01-19 16:00:00| 181|            159.35|   14|
|2020-01-14 21:00:00| 129|            145.52|   12|
|2020-01-03 11:00:00| 191|             98.75|    4|
|2020-01-25 14:00:00| 152|              96.1|    7|
|2020-01-15 08:00:00| 101|             31.79|    1|
|2020-01-20 13:00:00|  74|            592.28|   50|
|2020-01-22 21:00:00| 134|            252.32|   17|
|2020-01-02 08:00:00|  74|           1187.42|   96|
|2020-01-30 19:00:00|  61|            280.24|   13|
|2020-01-04 11:00:00|   3|116.27000000000001|    2|
|2020-01-14 16:00:00|  52|            191.96|   10|
|2020-01-27 11:00:00|  22|            121.66|    3|
|2020-01-18 

In [21]:
df_result.write.parquet('tmp/green-revenue', mode='overwrite')

In [22]:
columns = ['VendorID', 'lpep_pickup_datetime', 'PULocationID', 'DOLocationID', 'trip_distance']

duration_rdd = df_green \
    .select(columns) \
    .rdd

In [23]:
import pandas as pd

In [24]:
rows = duration_rdd.take(10)

In [25]:
df = pd.DataFrame(rows, columns=columns)

In [26]:
columns

['VendorID',
 'lpep_pickup_datetime',
 'PULocationID',
 'DOLocationID',
 'trip_distance']

In [27]:
#model = ...

def model_predict(df):
#     y_pred = model.predict(df)
    y_pred = df.trip_distance * 5
    return y_pred

In [28]:
def apply_model_in_batch(rows):
    df = pd.DataFrame(rows, columns=columns)
    predictions = model_predict(df)
    df['predicted_duration'] = predictions

    for row in df.itertuples():
        yield row

In [29]:
df_predicts = duration_rdd \
    .mapPartitions(apply_model_in_batch)\
    .toDF() \
    .drop('Index')

In [30]:
df_predicts.show()

+--------+--------------------+------------+------------+-------------+------------------+
|VendorID|lpep_pickup_datetime|PULocationID|DOLocationID|trip_distance|predicted_duration|
+--------+--------------------+------------+------------+-------------+------------------+
|     2.0|                  []|          41|          41|         0.78|3.9000000000000004|
|     2.0|                  []|         173|          70|         0.98|               4.9|
|     2.0|                  []|          74|         236|          2.7|              13.5|
|     1.0|                  []|          25|          66|          0.8|               4.0|
|     NaN|                  []|         259|          51|         2.33|             11.65|
|     2.0|                  []|         177|         188|         2.62|13.100000000000001|
|     2.0|                  []|          65|          97|         1.13|5.6499999999999995|
|     NaN|                  []|         165|         123|         1.36| 6.800000000000001|

In [31]:
df_predicts.select('predicted_duration').show()

+------------------+
|predicted_duration|
+------------------+
|3.9000000000000004|
|               4.9|
|              13.5|
|               4.0|
|             11.65|
|13.100000000000001|
|5.6499999999999995|
| 6.800000000000001|
|             55.75|
|               8.9|
|               5.0|
|             13.75|
|               5.5|
|             19.05|
|              9.25|
|              45.7|
|               5.2|
| 5.699999999999999|
|              5.75|
|4.6000000000000005|
+------------------+
only showing top 20 rows



In [31]:
spark.stop()